# 7. Visualização Cartográfica

_“A criação de mapas é um dos atividades intelectuais mais antigos da humanidade e também uma das mais complexas, combinando teoria científica, representação gráfica, fatos geográficos e considerações práticas de diversas maneiras.”_ &mdash; [H. J. Steward](https://books.google.com/books?id=cVy1Ms43fFYC)

A cartografia &ndash; o estudo e a prática da criação de mapas &ndash; tem uma história rica em descobertas e designs ao longo dos séculos. A visualização cartográfica utiliza técnicas de mapeamento para representar dados contendo informações espaciais, como localizações, rotas ou trajetórias na superfície da Terra.

<div style="float: right; margin-left: 1em; margin-top: 1em;"><img width="300px" src="https://gist.githubusercontent.com/jheer/c90d582ef5322582cf4960ec7689f6f6/raw/8dc92382a837ccc34c076f4ce7dd864e7893324a/latlon.png" /></div>


Ao aproximarmos a Terra para uma esfera, podemos denotar posições utilizando um sistema de coordenadas esféricas baseado em _latitude_ (ângulo em graus indo ao norte ou ao sul do _equador_) e _longitude_ (ângulo em graus que especifica a posição leste-oeste). Nesse sistema, um _paralelo_ é um círculo de latitude constante, enquanto um _meridiano_ é um círculo de longitude constante. O [_Meridiano de Greenwich_](https://en.wikipedia.org/wiki/Prime_meridian) está localizado na longitude 0° e, por convenção, passa pelo Observatório Real de Greenwich, na Inglaterra.

Para "achatar" uma esfera tridimensional em um plano bidimensional, é necessário aplicar uma [projeção](https://en.wikipedia.org/wiki/Map_projection) que mapeie pares de coordenadas (`longitude`, `latitude`) para (`x`, `y`). De forma semelhante às [escalas](https://github.com/uwdata/visualization-curriculum/blob/master/altair_scales_axes_legends.ipynb), as projeções transformam um domínio de dados (posição espacial) em um intervalo visual (posição em pixels). No entanto, as escalas que vimos até agora aceitam um domínio unidimensional, enquanto as projeções cartográficas são inerentemente bidimensionais.

Neste notebook, introduziremos os conceitos básicos da criação de mapas e da visualização de dados espaciais com Altair, incluindo:

- Formatos de dados para representar elementos geográficos,
- Técnicas de geo-visualização, como mapas de pontos, símbolos e mapas coropléticos, e
- Uma revisão das projeções cartográficas mais comuns.

_Este notebook faz parte do [currículo de visualização de dados](https://github.com/uwdata/visualization-curriculum)._


In [ ]:
import pandas as pd
import altair as alt
from vega_datasets import data

## 7.1. Dados Geográficos: GeoJSON e TopoJSON

Até agora, trabalhamos com conjuntos de dados nos formatos JSON e CSV, que correspondem a tabelas de dados compostas por linhas (registros) e colunas (campos). No entanto, para representar regiões geográficas (países, estados, _etc._) e trajetórias (rotas de voo, linhas de metrô, _etc._), precisamos expandir nosso repertório com formatos adicionais projetados para suportar geometrias complexas.

O [GeoJSON](https://en.wikipedia.org/wiki/GeoJSON) modela elementos geográficos dentro de um formato especializado de JSON. Um `feature` do GeoJSON pode incluir dados geométricos &ndash; como coordenadas de `longitude` e `latitude` que definem o contorno de um país &ndash; além de atributos adicionais dos dados.

Aqui está um objeto `feature` do GeoJSON que representa o contorno do estado do Colorado, nos Estados Unidos:


~~~ json
{
  "type": "Feature",
  "id": 8,
  "properties": {"name": "Colorado"},
  "geometry": {
    "type": "Polygon",
    "coordinates": [
      [[-106.32056285448942,40.998675790862656],[-106.19134826714341,40.99813863734313],[-105.27607827344248,40.99813863734313],[-104.9422739227986,40.99813863734313],[-104.05212898774828,41.00136155846029],[-103.57475287338661,41.00189871197981],[-103.38093099236758,41.00189871197981],[-102.65589358559272,41.00189871197981],[-102.62000064466328,41.00189871197981],[-102.052892177978,41.00189871197981],[-102.052892177978,40.74889940428302],[-102.052892177978,40.69733266640851],[-102.052892177978,40.44003613055551],[-102.052892177978,40.3492571857556],[-102.052892177978,40.00333031918079],[-102.04930288388505,39.57414465707943],[-102.04930288388505,39.56823596836465],[-102.0457135897921,39.1331416175485],[-102.0457135897921,39.0466599009048],[-102.0457135897921,38.69751011321283],[-102.0457135897921,38.61478847120581],[-102.0457135897921,38.268861604631],[-102.0457135897921,38.262415762396685],[-102.04212429569915,37.738153927339205],[-102.04212429569915,37.64415206142214],[-102.04212429569915,37.38900413964724],[-102.04212429569915,36.99365914927603],[-103.00046581851544,37.00010499151034],[-103.08660887674611,37.00010499151034],[-104.00905745863294,36.99580776335414],[-105.15404227428235,36.995270609834606],[-105.2222388620483,36.995270609834606],[-105.7175614468747,36.99580776335414],[-106.00829426840322,36.995270609834606],[-106.47490250048605,36.99365914927603],[-107.4224761410235,37.00010499151034],[-107.48349414060355,37.00010499151034],[-108.38081766383978,36.99903068447129],[-109.04483707103458,36.99903068447129],[-109.04483707103458,37.484617466122884],[-109.04124777694163,37.88049961001363],[-109.04124777694163,38.15283644441336],[-109.05919424740635,38.49983761802722],[-109.05201565922046,39.36680339854235],[-109.05201565922046,39.49786885730673],[-109.05201565922046,39.66062637372313],[-109.05201565922046,40.22248895514744],[-109.05201565922046,40.653823231326896],[-109.05201565922046,41.000287251421234],[-107.91779872584989,41.00189871197981],[-107.3183866123281,41.00297301901887],[-106.85895696843116,41.00189871197981],[-106.32056285448942,40.998675790862656]]
    ]
  }
}
~~~

O `feature` inclui um objeto `properties`, que pode inlcuir qualquer número de campos de dados, além de um objeto `geometry`, que, neste caso, contém um único polígono formado por coordenadas `[longitude, latitude]` que definem o contorno do estado. As coordenadas continuam à direita por um bom trecho, caso queira continuar rolando o scroll do mouse...

Para saber mais sobre os detalhes técnicos do GeoJSON, consulte a [especificação oficial do GeoJSON](http://geojson.org/) ou leia a [introdução útil de Tom MacWright](https://macwright.org/2015/03/23/geojson-second-bite).


Uma desvantagem do GeoJSON como formato de armazenamento é que ele pode ser redundante, resultando em arquivos de tamanhos maiores. Considere: o Colorado compartilha fronteiras com seis outros estados (sete, se incluirmos o ponto de contato com o Arizona). Ao invés de usar listas de coordenadas sobrepostas para cada um desses estados, uma abordagem mais esperta é codificar as fronteiras compartilhadas apenas uma vez, representando a _topologia_ das regiões geográficas. Felizmente, é exatamente isso que o formato [TopoJSON](https://github.com/topojson/topojson/blob/master/README.md) faz!


Vamos carregar um arquivo TopoJSON de países do mundo (numa resolução de 110 metros):

In [ ]:
world = data.world_110m.url
world

In [ ]:
world_topo = data.world_110m()

In [ ]:
world_topo.keys()

In [ ]:
world_topo['type']

In [ ]:
world_topo['objects'].keys()

_Inspecione o dicionário TopoJSON `world_topo` acima para ver seu conteúdo._

Nos dados acima, a propriedade `objects` indica os elementos nomeados que podemos extrair: geometrias para todos os `countries` (países) ou um único polígono representando toda a superfície terrestre (`land`). Qualquer um desses elementos pode ser convertido para dados GeoJSON que podemos futuramente visualizar.

Como o TopoJSON é um formato especializado, precisamos instruir o Altair a interpretar corretamente o formato TopoJSON, indicando qual objeto nomeado queremos extrair da topologia. O código a seguir indica que queremos extrair os elementos GeoJSON do dataset `world` para o objeto `countries`:

~~~ js
alt.topo_feature(world, 'countries')
~~~

Essa chamada ao método `alt.topo_feature` se expande para o seguinte JSON do Vega-Lite:

~~~ json
{
  "values": world,
  "format": {"type": "topojson", "feature": "countries"}
}
~~~

Agora que podemos carregar dados geográficos, estamos prontos para começar a fazer mapas!


## 7.2. Marcas de Formato Geográfico

Para visualizar dados geográficos, o Altair fornece o tipo de marca `geoshape`. Para criar um mapa básico, podemos criar uma marca `geoshape` e passá-la para nossos dados TopoJSON, que será, então, convertido em recursos GeoJSON, um para cada país do mundo

In [ ]:
alt.Chart(alt.topo_feature(world, 'countries')).mark_geoshape()

No exemplo acima, o Altair aplica uma cor azul padrão e usa a projeção cartográfica padrão (`mercator`). Podemos personalizar as cores e a largura dos contornos usando propriedades padrão das marcas. Usando o método `project`, podemos definir nossa própria projeção de mapa:


In [ ]:
alt.Chart(alt.topo_feature(world, 'countries')).mark_geoshape(
    fill='#2a1d0c', stroke='#706545', strokeWidth=0.5
).project(
    type='mercator'
)

Por padrão, o Altair ajusta automaticamente a projeção para que todos os dados caibam dentro da largura e altura do gráfico. Podemos também especificar parâmetros de projeção, como `scale` (nível de zoom) e `translate` (deslocamento), para personalizar as configurações da projeção. No exemplo abaixo, ajustamos os parâmetros `scale` e `translate` para focar na Europa:


In [ ]:
alt.Chart(alt.topo_feature(world, 'countries')).mark_geoshape(
    fill='#2a1d0c', stroke='#706545', strokeWidth=0.5
).project(
    type='mercator', scale=400, translate=[100, 550]
)

_Observe como a resolução de 110m dos dados se torna aparente nessa escala. Para ver linhas costeiras e fronteiras com mais detalhes, precisamos de um arquivo de entrada com geometrias mais refinadas. Ajuste os parâmetros `scale` e `translate` para focar o mapa em outras regiões!_


Até agora, nosso mapa exibe apenas países. Usando o operador `layer`, podemos combinar múltiplos elementos de mapa. O Altair inclui _geradores de dados_ que podemos utilizar para criar camadas de mapa adicionais:

- O gerador de esfera (`{'sphere': True}`) fornece uma representação GeoJSON da esfera completa da Terra. Podemos criar uma marca `geoshape` adicional para preencher o formato da Terra como uma camada de fundo.
- O gerador de graticulado (`{'graticule': ...}`) cria um elemento GeoJSON representando um _reticulado_ ****: uma grade formada por linhas de latitude e longitude. O reticulado padrão possui meridianos e paralelos a cada 10° entre ±80° de latitude. Para as regiões polares, há meridianos a cada 90°. Esses ajustes podem ser personalizados usando as propriedades `stepMinor` e `stepMajor`.

Vamos combinar as camadas de esfera, reticulado e países em uma especificação de mapa reutilizável:


In [ ]:
map = alt.layer(
    # use a esfera da Terra como camada base
    alt.Chart({'sphere': True}).mark_geoshape(
        fill='#e6f3ff'
    ),
    # adicione uma gratícula para linhas de referência geográfica
    alt.Chart({'graticule': True}).mark_geoshape(
        stroke='#ffffff', strokeWidth=1
    ),
    # e então os países do mundo
    alt.Chart(alt.topo_feature(world, 'countries')).mark_geoshape(
        fill='#2a1d0c', stroke='#706545', strokeWidth=0.5
    )
).properties(
    width=600,
    height=400
)

Podemos estender o mapa aplicando uma projeção desejada e desenhando o resultado. Aqui, utilizamos a [projeção Natural da Terra](https://en.wikipedia.org/wiki/Natural_Earth_projection). A camada _sphere_ fornece o fundo azul claro, enquanto a camada _graticule_ fornece as linhas de referência geográficas em branco.


In [ ]:
map.project(
    type='naturalEarth1', scale=110, translate=[300, 200]
).configure_view(stroke=None)

## 7.3. Mapas de Ponto

Além dos dados _geométricos_ fornecidos por arquivos GeoJSON ou TopoJSON, muitos conjuntos de dados tabulares incluem informações geográficas na forma de campos de coordenadas `longitude` e `latitude`, ou referências a regiões geográficas, como nomes de países, estados, códigos postais, _etc._ Esses dados podem ser convertidos em coordenadas usando um [serviço de geocodificação](https://en.wikipedia.org/wiki/Geocoding).  

Em alguns casos, os dados de localização são tão ricos que conseguimos identificar padrões significativos apenas projetando os pontos de dados &mdash; sem a necessidade de um mapa de base!  

Vamos analisar um conjunto de dados com códigos postais de 5 dígitos dos Estados Unidos, incluindo as coordenadas `longitude` e `latitude` de cada agência dos correios, além do campo `zip_code`.


In [ ]:
zipcodes = data.zipcodes.url
zipcodes

Podemos visualizar cada agência dos correios usando uma pequena marca `square` (de 1 pixel). No entanto, para definir as posições, _não_ utilizamos os canais `x` e `y`. _Por que isso acontece?_

Embora as projeções cartográficas mapeiem coordenadas (`longitude`, `latitude`) para (`x`, `y`), esse mapeamento pode ser feito de diversas maneiras. Não há garantia, por exemplo, de que `longitude` → `x` e `latitude` → `y`!  

Em vez disso, o Altair inclui canais especiais de codificação `longitude` e `latitude` para lidar com coordenadas geográficas. Esses canais indicam quais campos de dados devem ser mapeados para `longitude` e `latitude`, e então aplicam uma projeção para converter essas coordenadas em posições (`x`, `y`) no gráfico.


In [ ]:
alt.Chart(zipcodes).mark_square(
    size=1, opacity=1
).encode(
    longitude='longitude:Q', # aplique o campo chamado 'longitude' ao canal de longitude
    latitude='latitude:Q'    # aplique o campo chamado 'latitude' ao canal de latitude
).projeto(
    type='albersUsa'
).properties(
    width=900,
    height=500
).configure_view(
    stroke=None
)

_Ao plotar apenas os códigos postais, conseguimos visualizar o contorno dos Estados Unidos e identificar padrões significativos na densidade das agências dos correios, sem a necessidade de um mapa de base ou elementos adicionais de referência!_  

Usamos a projeção `albersUsa`, que faz algumas adaptações na geometria real da Terra, posicionando versões redimensionadas do Alasca e do Havaí no canto inferior esquerdo. Como não especificamos os parâmetros `scale` ou `translate` da projeção, o Altair os ajusta automaticamente para encaixar os dados visualizados.  

Agora podemos explorar mais perguntas sobre nosso conjunto de dados. Por exemplo, há algum padrão na alocação dos códigos postais? Para avaliar essa questão, podemos adicionar uma codificação de cor baseada no primeiro dígito do código postal.  

Primeiro, aplicamos uma transformação `calculate` para extrair o primeiro dígito e, em seguida, utilizamos o canal de cor para representá-lo visualmente:


In [ ]:
alt.Chart(zipcodes).transform_calculate(
    digit='datum.zip_code[0]'
).mark_square(
    size=2, opacity=1
).encode(
    longitude='longitude:Q',
    latitude='latitude:Q',
    color='digit:N'
).project(
    type='albersUsa'
).properties(
    width=900,
    height=500
).configure_view(
    stroke=None
)

_Para dar zoom um dígito específico, adicione uma transformação de filtro para limitar os dados exibidos! Experimente incluir uma [seleção interativa](https://github.com/uwdata/visualization-curriculum/blob/master/altair_interaction.ipynb) para filtrar um único dígito e atualizar dinamicamente o mapa. E lembre-se de usar strings (\`'1'\`) em vez de números (\`1\`) ao filtrar valores de dígitos!_  

(Este exemplo foi inspirado na clássica visualização [zipdecode](https://benfry.com/zipdecode/) de Ben Fry!)  

Podemos também nos perguntar o que a _sequência_ dos códigos postais pode indicar. Uma maneira de explorar essa questão é conectar cada código postal consecutivo usando uma marca `line`, como na visualização [ZipScribble](https://eagereyes.org/zipscribble-maps/united-states) de Robert Kosara:


In [ ]:
alt.Chart(zipcodes).transform_filter(
    '-150 < datum.longitude && 22 < datum.latitude && datum.latitude < 55'
).transform_calculate(
    digit='datum.zip_code[0]'
).mark_line(
    strokeWidth=0.5
).encode(
    longitude='longitude:Q',
    latitude='latitude:Q',
    color='digit:N',
    order='zip_code:O'
).project(
    type='albersUsa'
).properties(
    width=900,
    height=500
).configure_view(
    stroke=None
)

_Agora podemos observar como os códigos postais se agrupam ainda mais em áreas menores, indicando uma alocação hierárquica dos códigos por localização, embora com alguma variabilidade notável dentro de clusters locais._  

Se você prestou atenção nos nossos mapas anteriores, pode ter notado que alguns códigos postais estão sendo plotados no canto superior esquerdo! Eles correspondem a locais como Porto Rico ou Samoa Americana, que possuem códigos postais dos EUA, mas são mapeados para coordenadas `null` (`0`, `0`) pela projeção `albersUsa`. Além disso, Alasca e Havaí podem complicar a visualização das conexões entre os segmentos de linha.  

Para lidar com isso, o código acima inclui um filtro adicional que remove pontos fora dos intervalos escolhidos de `longitude` e `latitude`.  

_Remova o filtro acima para ver o que acontece!_


## 7.4. Mapas de Símbolos

Agora vamos combinar um mapa base e os dados plotados como camadas separadas. Vamos examinar a rede de voos comerciais dos EUA, considerando tanto os aeroportos quanto as rotas de voo. Para isso, precisaremos de três conjuntos de dados.  
Para o nosso mapa base, usaremos um arquivo TopoJSON dos Estados Unidos com resolução de 10m, contendo elementos para `estados` ou `condados`:


In [ ]:
usa = data.us_10m.url
usa

Para os aeroportos, usaremos um conjunto de dados com campos para as coordenadas `longitude` e `latitude` de cada aeroporto, além do código do aeroporto `iata` &mdash; por exemplo, `'SEA'` para o [Aeroporto Internacional de Seattle-Tacoma](https://en.wikipedia.org/wiki/Seattle%E2%80%93Tacoma_International_Airport).


In [ ]:
airports = data.airports.url
airports

Por fim, vamos usar um dataset de rotas de voo, que contém campos `origin` e `destination` com os códigos IATA para os aeroportos correspondentes:

In [ ]:
flights = data.flights_airport.url
flights

Vamos começar criando um mapa base usando a projeção `albersUsa` e adicionar uma camada que exibe marcas `circle` para cada aeroporto:

In [ ]:
alt.layer(
    alt.Chart(alt.topo_feature(usa, 'states')).mark_geoshape(
        fill='#ddd', stroke='#fff', strokeWidth=1
    ),
    alt.Chart(airports).mark_circle(size=9).encode(
        latitude='latitude:Q',
        longitude='longitude:Q',
        tooltip='iata:N'
    )
).project(
    type='albersUsa'
).properties(
    width=900,
    height=500
).configure_view(
    stroke=None
)

_São muitos aeroportos! Obviamente, nem todos são grandes hubs._  

Semelhante ao nosso dataset de códigos postais, os dados dos aeroportos incluem pontos que estão fora dos Estados Unidos continentais. Por isso, novamente vemos pontos no canto superior esquerdo. Talvez queiramos filtrar esses pontos, mas para isso precisamos primeiro entender mais sobre eles.  

_Atualize a projeção do mapa acima para `albers` &ndash; contornando o comportamento peculiar do `albersUsa` &ndash; para que as localizações reais desses pontos adicionais sejam reveladas!_  

Agora, em vez de mostrar todos os aeroportos de forma indiferenciada, vamos identificar os grandes hubs considerando o número total de rotas que se originam em cada aeroporto. Vamos usar o dataset `routes` como nossa principal fonte de dados: ele contém uma lista de rotas de voo que podemos agrupar para contar o número de rotas para cada aeroporto `origin`.  

No entanto, o dataset `routes` não inclui as _localizações_ dos aeroportos! Para complementar os dados de `routes` com as localizações, precisamos de uma nova transformação de dados: `lookup`. A transformação `lookup` pega um valor de campo em um dataset principal e usa esse valor como uma _chave_ para procurar informações relacionadas em outra tabela. Neste caso, queremos combinar o código do aeroporto `origin` do nosso dataset `routes` com o campo `iata` do dataset `airports`, e então extrair os campos correspondentes de `latitude` e `longitude`.


In [ ]:
alt.layer(
    alt.Chart(alt.topo_feature(usa, 'states')).mark_geoshape(
        fill='#ddd', stroke='#fff', strokeWidth=1
    ),
    alt.Chart(flights).mark_circle().transform_aggregate(
        groupby=['origin'],
        routes='count()'
    ).transform_lookup(
        lookup='origin',
        from_=alt.LookupData(data=airports, key='iata',
                             fields=['state', 'latitude', 'longitude'])
    ).transform_filter(
        'datum.state !== "PR" && datum.state !== "VI"'
    ).encode(
        latitude='latitude:Q',
        longitude='longitude:Q',
        tooltip=['origin:N', 'routes:Q'],
        size=alt.Size('routes:Q', scale=alt.Scale(range=[0, 1000]), legend=None),
        order=alt.Order('routes:Q', sort='descending')
    )
).project(
    type='albersUsa'
).properties(
    width=900,
    height=500
).configure_view(
    stroke=None
)

_Quais aeroportos dos EUA têm o maior número de rotas de saída?_  

Agora que podemos ver os aeroportos, talvez queiramos interagir com eles para entender melhor a estrutura da rede de tráfego aéreo. Podemos adicionar uma camada de marca `rule` para representar os caminhos de aeroportos `origin` para aeroportos `destination`, o que requer duas transformações `lookup` para recuperar as coordenadas de cada ponto de extremidade. Além disso, podemos usar uma seleção `single` para filtrar essas rotas, de forma que apenas as rotas originadas no aeroporto atualmente selecionado sejam mostradas.

_Começando a partir do mapa estático acima, você consegue construir uma versão interativa? Fique à vontade para pular o código abaixo e interagir com o mapa interativo primeiro, e depois pense em como construir isso por conta própria!_


In [ ]:
# seleção interativa para aeroporto de origem
# selecione o aeroporto mais próximo com o cursor do mouse
origin = alt.selection_single(
    on='mouseover', nearest=True,
    fields=['origin'], empty='none'
)

# referência de dados compartilhada para transformações de pesquisa
foreign = alt.LookupData(data=airports, key='iata',
                         fields=['latitude', 'longitude'])
    
alt.layer(
    # mapa básico dos Estados Unidos
    alt.Chart(alt.topo_feature(usa, 'states')).mark_geoshape(
        fill='#ddd', stroke='#fff', strokeWidth=1
    ),
    # linhas de rota do aeroporto de origem selecionado até os aeroportos de destino
    alt.Chart(flights).mark_rule(
        color='#000', opacity=0.35
    ).transform_filter(
        origin # filtrar apenas para a origem selecionada
    ).transform_lookup(
        lookup='origin', from_=foreign # origem lat/lon
    ).transform_lookup(
        lookup='destination', from_=foreign, as_=['lat2', 'lon2'] # latitude/longo destino
    ).encode(
        latitude='latitude:Q',
        longitude='longitude:Q',
        latitude2='lat2',
        longitude2='lon2',
    ),
    # tamanho dos aeroportos por número de rotas de saída
    # 1. conjunto de dados agregados de voos-aeroporto
    # 2. pesquisar dados de localização do conjunto de dados de aeroportos
    # 3. remover Porto Rico (PR) e Ilhas Virgens (VI)
    alt.Chart(flights).mark_circle().transform_aggregate(
        groupby=['origin'],
        routes='count()'
    ).transform_lookup(
        lookup='origin',
        from_=alt.LookupData(data=airports, key='iata',
                             fields=['state', 'latitude', 'longitude'])
    ).transform_filter(
        'datum.state !== "PR" && datum.state !== "VI"'
    ).add_selection(
        origin
    ).encode(
        latitude='latitude:Q',
        longitude='longitude:Q',
        tooltip=['origin:N', 'routes:Q'],
        size=alt.Size('routes:Q', scale=alt.Scale(range=[0, 1000]), legend=None),
        order=alt.Order('routes:Q', sort='descending') # coloque círculos menores no topo
    )
).project(
    type='albersUsa'
).properties(
    width=900,
    height=500
).configure_view(
    stroke=None
)

Passe o mouse sobre o mapa para conhecer a rede de voos!

## 7.5. Mapas Coropléticos

Um [mapa coroplético](https://en.wikipedia.org/wiki/Choropleth_map) usa regiões sombreadas ou texturizadas para visualizar valores de dados. Mapas de símbolos dimensionados são frequentemente mais precisos para ler, pois as pessoas tendem a ser melhores em estimar as diferenças proporcionais entre a área dos círculos do que entre os tons de cor. No entanto, mapas choropléticos são populares na prática e particularmente úteis quando muitos símbolos se tornam visualmente sobrecarregados.

Por exemplo, enquanto os Estados Unidos têm apenas 50 estados, eles têm milhares de condados dentro desses estados. Vamos construir um mapa choroplético da taxa de desemprego por condado, no ano de recessão de 2008. Em alguns casos, arquivos de entrada em GeoJSON ou TopoJSON podem incluir dados estatísticos que podemos visualizar diretamente. No entanto, neste caso, temos dois arquivos: nosso arquivo TopoJSON que inclui as características dos limites dos condados (`usa`), e um arquivo de texto separado que contém as estatísticas de desemprego:


In [ ]:
unemp = data.unemployment.url
unemp

Para integrar nossas fontes de dados, precisaremos novamente usar a transformação `lookup`, complementando nossos dados `geoshape` baseados em TopoJSON com as taxas de desemprego. Podemos então criar um mapa que inclui uma codificação de `color` para o campo `rate` obtido pela busca.


In [ ]:
alt.Chart(alt.topo_feature(usa, 'counties')).mark_geoshape(
    stroke='#aaa', strokeWidth=0.25
).transform_lookup(
    lookup='id', from_=alt.LookupData(data=unemp, key='id', fields=['rate'])
).encode(
    alt.Color('rate:Q',
              scale=alt.Scale(domain=[0, 0.3], clamp=True), 
              legend=alt.Legend(format='%')),
    alt.Tooltip('rate:Q', format='.0%')
).project(
    type='albersUsa'
).properties(
    width=900,
    height=500
).configure_view(
    stroke=None
)

*Examine as taxas de desemprego por condado. Valores mais altos em Michigan podem estar relacionados à indústria automotiva. Condados nas regiões das [Great Plains](https://en.wikipedia.org/wiki/Great_Plains) e nos estados das Montanhas exibem tanto taxas baixas **quanto** altas. Será que essa variação é significativa, ou é possivelmente um [artifício de amostras menores](https://medium.com/@uwdata/surprise-maps-showing-the-unexpected-e92b67398865)? Para explorar mais, tente alterar o domínio superior da escala (por exemplo, para `0.2`) para ajustar o mapeamento das cores.*

Uma preocupação central para mapas choropléticos é a escolha das cores. Acima, usamos o esquema padrão `'yellowgreenblue'` do Altair para mapas de calor. Abaixo, comparamos outros esquemas, incluindo um esquema _sequencial de uma única tonalidade_ (`teals`) que varia apenas na luminância, um esquema _sequencial multicolorido_ (`viridis`) que varia tanto em luminância quanto em tonalidade, e um esquema _divergente_ (`blueorange`) que usa um ponto médio branco:


In [ ]:
# função utilitária para gerar uma especificação de mapa para um esquema de cores fornecido
def map_(scheme):
    return alt.Chart().mark_geoshape().project(type='albersUsa').encode(
        alt.Color('rate:Q', scale=alt.Scale(scheme=scheme), legend=None)
    ).properties(width=305, height=200)

alt.hconcat(
    map_('tealblues'), map_('viridis'), map_('blueorange'),
    data=alt.topo_feature(usa, 'counties')
).transform_lookup(
    lookup='id', from_=alt.LookupData(data=unemp, key='id', fields=['rate'])
).configure_view(
    stroke=None
).resolve_scale(
    color='independent'
)

_Qual esquema de cor você acha que é mais eficaz? Por que? Modifique os mapas acima e use outros esquemas disponíveis, como descrito na [documentação dos Esquemas de Cor do Vega](https://vega.github.io/vega/docs/schemes/)._

## 7.6. Projeções Cartográficas

Agora que já temos alguma experiência criando mapas, vamos dar uma olhada mais de perto nas projeções cartográficas. Como explicado pela [Wikipedia](https://en.wikipedia.org/wiki/Map_projection),

> _Todas as projeções de mapas necessariamente distorcem a superfície de alguma forma. Dependendo do objetivo do mapa, algumas distorções são aceitáveis e outras não; portanto, diferentes projeções de mapas existem para preservar algumas propriedades do corpo esférico à custa de outras propriedades._

Algumas das propriedades que podemos considerar incluem:

- _Área_: A projeção distorce os tamanhos das regiões?
- _Rumo_: Uma linha reta corresponde a uma direção constante de viagem?
- _Distância_: Linhas de comprimento igual correspondem a distâncias iguais na esfera?
- _Forma_: A projeção preserva as relações espaciais (ângulos) entre pontos?

Selecionar uma projeção apropriada depende, portanto, do caso de uso do mapa. Por exemplo, se estamos avaliando o uso da terra e a extensão da terra é importante, poderíamos escolher uma projeção que preserve a área. Se quisermos visualizar ondas de choque emanando de um terremoto, podemos focar o mapa no epicentro do terremoto e preservar as distâncias a partir daquele ponto. Ou, se quisermos auxiliar na navegação, a preservação do rumo e da forma pode ser mais importante.

Também podemos caracterizar as projeções em termos da _superfície de projeção_. Projeções cilíndricas, por exemplo, projetam os pontos da superfície da esfera em torno de um cilindro; o cilindro "desenrolado" então fornece nosso mapa. Como descrevemos abaixo, podemos projetar alternativamente sobre a superfície de um cone (projeções cônicas) ou diretamente sobre um plano plano (projeções azimutais).

*Vamos primeiro construir nossa intuição interagindo com uma variedade de projeções! **[Abra o notebook online de Projeções Cartográficas do Vega-Lite](https://observablehq.com/@vega/vega-lite-cartographic-projections).** Use os controles naquela página para selecionar uma projeção e explorar parâmetros de projeção, como o `scale` (zoom) e a tradução x/y (movimento). Os controles de rotação ([yaw, pitch, roll](https://en.wikipedia.org/wiki/Aircraft_principal_axes)) determinam a orientação do globo em relação à superfície sobre a qual ele está sendo projetado.*


### 7.6.1. Uma Visita aos Tipos Específicos de Projeção

[**Projeções cilíndricas**](https://en.wikipedia.org/wiki/Map_projection#Cylindrical) mapeiam a esfera para um cilindro ao redor, depois desenrolam o cilindro. Se o eixo principal do cilindro estiver orientado no sentido norte-sul, os meridianos são mapeados para linhas retas. As projeções [pseudo-cilíndricas](https://en.wikipedia.org/wiki/Map_projection#Pseudocylindrical) representam um meridiano central como uma linha reta, com outros meridianos "curvando-se" para longe do centro.


In [ ]:
minimap = map.properties(width=225, height=225)
alt.hconcat(
    minimap.project(type='equirectangular').properties(title='equirectangular'),
    minimap.project(type='mercator').properties(title='mercator'),
    minimap.project(type='transverseMercator').properties(title='transverseMercator'),
    minimap.project(type='naturalEarth1').properties(title='naturalEarth1')
).properties(spacing=10).configure_view(stroke=None)

- [Equiretangular](https://en.wikipedia.org/wiki/Equirectangular_projection) (`equirectangular`): Escala os valores das coordenadas `lat` e `lon` diretamente.
- [Mercator](https://en.wikipedia.org/wiki/Mercator_projection) (`mercator`): Projeta em um cilindro, usando `lon` diretamente, mas sujeitando `lat` a uma transformação não linear. Linhas retas preservam as direções constantes da bússola ([linhas de rumo](https://en.wikipedia.org/wiki/Rhumb_line)), tornando essa projeção bem adequada para navegação. No entanto, áreas no extremo norte ou sul podem ser grandemente distorcidas.
- [Mercator Transversal](https://en.wikipedia.org/wiki/Transverse_Mercator_projection) (`transverseMercator`): Uma projeção mercator, mas com o cilindro de contorno rotacionado para um eixo transversal. Enquanto a projeção padrão de Mercator tem maior precisão ao longo do equador, a projeção Mercator Transversal é mais precisa ao longo do meridiano central.
- [Natural Earth](https://en.wikipedia.org/wiki/Natural_Earth_projection) (`naturalEarth1`): Uma projeção pseudo-cilíndrica projetada para mostrar o mundo inteiro em uma única visão.
<br/><br/>


[**Projeções cônicas**](https://en.wikipedia.org/wiki/Map_projection#Conic) mapeiam a esfera para um cone e depois desenrolam o cone sobre o plano. As projeções cônicas são configuradas por dois _paralelos padrão_, que determinam onde o cone intersecta a esfera.


In [ ]:
minimap = map.properties(width=180, height=130)
alt.hconcat(
    minimap.project(type='conicEqualArea').properties(title='conicEqualArea'),
    minimap.project(type='conicEquidistant').properties(title='conicEquidistant'),
    minimap.project(type='conicConformal', scale=35, translate=[90,65]).properties(title='conicConformal'),
    minimap.project(type='albers').properties(title='albers'),
    minimap.project(type='albersUsa').properties(title='albersUsa')
).properties(spacing=10).configure_view(stroke=None)

- [Área Cônica Igual](https://en.wikipedia.org/wiki/Albers_projection) (`conicEqualArea`): Projeção cônica que preserva a área. A forma e a distância não são preservadas, mas a precisão é razoável dentro dos paralelos padrão.
- [Cônica Equidistante](https://en.wikipedia.org/wiki/Equidistant_conic_projection) (`conicEquidistant`): Projeção cônica que preserva a distância ao longo dos meridianos e paralelos padrão.
- [Cônica Conformal](https://en.wikipedia.org/wiki/Lambert_conformal_conic_projection) (`conicConformal`): Projeção cônica que preserva a forma (ângulos locais), mas não a área ou a distância.
- [Albers](https://en.wikipedia.org/wiki/Albers_projection) (`albers`): Uma variante da projeção cônica igual área com paralelos padrão otimizados para criar mapas dos Estados Unidos.
- [Albers EUA](https://en.wikipedia.org/wiki/Albers_projection) (`albersUsa`): Uma projeção híbrida para os 50 estados dos Estados Unidos da América. Essa projeção costura três projeções Albers com parâmetros diferentes para os EUA continentais, Alasca e Havai.


[**Projeções azimutais**](https://en.wikipedia.org/wiki/Map_projection#Azimuthal_%28projections_onto_a_plane%29) mapeie a esfera diretamente no plano.

In [ ]:
minimap = map.properties(width=180, height=180)
alt.hconcat(
    minimap.project(type='azimuthalEqualArea').properties(title='azimuthalEqualArea'),
    minimap.project(type='azimuthalEquidistant').properties(title='azimuthalEquidistant'),
    minimap.project(type='orthographic').properties(title='orthographic'),
    minimap.project(type='stereographic').properties(title='stereographic'),
    minimap.project(type='gnomonic').properties(title='gnomonic')
).properties(spacing=10).configure_view(stroke=None)

- [Área Azimutal Igual](https://en.wikipedia.org/wiki/Lambert_azimuthal_equal-area_projection) (`azimuthalEqualArea`): Projeta com precisão a área em todas as partes do globo, mas não preserva a forma (ângulos locais).
- [Azimutal Equidistante](https://en.wikipedia.org/wiki/Azimuthal_equidistant_projection) (`azimuthalEquidistant`): Preserva a distância proporcional do centro da projeção para todos os outros pontos no globo.
- [Ortográfica](https://en.wikipedia.org/wiki/Orthographic_projection_in_cartography) (`orthographic`): Projeta um hemisfério visível em um plano distante. Aproximadamente corresponde a uma visão da Terra do espaço.
- [Estereográfica](https://en.wikipedia.org/wiki/Stereographic_projection) (`stereographic`): Preserva a forma, mas não a área ou a distância.
- [Gnômica](https://en.wikipedia.org/wiki/Gnomonic_projection) (`gnomonic`): Projeta a superfície da esfera diretamente em um plano tangente. [Círculos máximos](https://en.wikipedia.org/wiki/Great_circle) ao redor da Terra são projetados para linhas retas, mostrando o caminho mais curto entre os pontos.


## 7.7. Coda: Manipulação de Dados Geográficos

Os exemplos acima utilizam a coleção vega-datasets, incluindo dados geométricos (TopoJSON) e tabulares (aeroportos, taxas de desemprego). Um desafio comum ao começar com visualização geográfica é coletar os dados necessários para sua tarefa. Existem muitos provedores de dados, incluindo serviços como o [United States Geological Survey](https://www.usgs.gov/products/data/all-data) e o [U.S. Census Bureau](https://www.census.gov/data/datasets.html).

Em muitos casos, você pode já ter dados existentes com um componente geográfico, mas precisar de medidas ou geometria adicionais. Para ajudar a começar, aqui está um fluxo de trabalho:

1. Visite o [Natural Earth Data](http://www.naturalearthdata.com/downloads/) e navegue para selecionar dados para regiões e resoluções de interesse. Baixe o(s) arquivo(s) zip correspondentes.
2. Vá para [MapShaper](https://mapshaper.org/) e arraste o arquivo zip baixado para a página. Revise os dados conforme desejado e depois "Exporte" os arquivos TopoJSON ou GeoJSON gerados.
3. Carregue os dados exportados do MapShaper para usar com o Altair!

Claro, existem muitas outras ferramentas — tanto de código aberto quanto proprietárias — para trabalhar com dados geográficos. Para mais sobre manipulação de dados geográficos e criação de mapas, veja a série de tutoriais de Mike Bostock sobre [Cartografia de Linha de Comando](https://medium.com/@mbostock/command-line-cartography-part-1-897aa8f8ca2c).


## 7.8. Resumo

Neste ponto, nós apenas começamos a explorar o mundo da criação de mapas. _(Você não esperava que um único notebook transmitisse séculos de aprendizado, esperava?)_ Por exemplo, deixamos de fora tópicos como [_cartogramas_](https://en.wikipedia.org/wiki/Cartogram) e a transmissão de [_topografia_](https://en.wikipedia.org/wiki/Topography) — como no iluminador livro de Imhof [_Cartographic Relief Presentation_](https://books.google.com/books?id=cVy1Ms43fFYC). No entanto, agora você deve estar bem equipado para criar uma rica variedade de geo-visualizações. Para mais, o livro de MacEachren [_How Maps Work: Representation, Visualization, and Design_](https://books.google.com/books?id=xhAvN3B0CkUC) fornece uma visão geral valiosa sobre a criação de mapas sob a perspectiva da visualização de dados.
